In [1]:
import os
import pickle
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import pearsonr

In [2]:
def age_debiasing(predicted_ages, chronological_ages):
    
    predicted_ages = predicted_ages.to_numpy()
    chronological_ages = chronological_ages.to_numpy()
    
    ones = np.ones(len(chronological_ages))
    chronological_ages_squared = np.square(chronological_ages)
    chronological_ages_confounds = np.array([ones, chronological_ages])
    
    ones_predicted = np.ones(len(predicted_ages))
    predicted_ages_squared = np.square(predicted_ages)
    predicted_ages_confounds = np.array([ones_predicted, predicted_ages])
    
    predicted_ages = np.reshape(predicted_ages, (len(predicted_ages), 1) )
    beta = np.dot( np.linalg.pinv(chronological_ages_confounds.T), predicted_ages )

    corrected_predicted_ages = (predicted_ages.flatten() - beta[0])/beta[1]
    
    return corrected_predicted_ages

In [3]:
T1_nonlinear="1"
T1_linear="2"
jacobian="56"
vbm="4"

T2_nonlinear="5"
T2_lesions="57"
swi="6"

rsfmri_0="7"
rsfmri_1="8"
rsfmri_2="9"
rsfmri_3="10"
rsfmri_4="11"
rsfmri_5="12"
rsfmri_6="13"
rsfmri_7="14"
rsfmri_8="15"
rsfmri_9="16"
rsfmri_10="17"
rsfmri_11="18"
rsfmri_12="19"
rsfmri_13="20"
rsfmri_14="21"
rsfmri_15="22"
rsfmri_16="23"
rsfmri_17="24"
rsfmri_18="25"
rsfmri_19="26"
rsfmri_20="27"
rsfmri_21="28"
rsfmri_22="29"
rsfmri_23="30"
rsfmri_24="31"

tfmri_1="32"
tfmri_2="33"
tfmri_5="34"

tfmri_c_1="35"
tfmri_c_2="36"
tfmri_c_5="37"

tracts="3"

tbss_FA_s="38"
tbss_ICVF_s="39"
tbss_ISOVF_s="40"
tbss_L1_s="41"
tbss_L2_s="42"
tbss_L3_s="43"
tbss_MD_s="44"
tbss_MO_s="45"
tbss_OD_s="46"

tbss_FA="47"
tbss_ICVF="48"
tbss_ISOVF="49"
tbss_L1="50"
tbss_L2="51"
tbss_L3="52"
tbss_MD="53"
tbss_MO="54"
tbss_OD="55"

dfs = [

T1_nonlinear,
T1_linear,
jacobian,
vbm,

T2_nonlinear,
T2_lesions,
swi,

rsfmri_0,
rsfmri_1,
rsfmri_2,
rsfmri_3,
rsfmri_4,
rsfmri_5,
rsfmri_6,
rsfmri_7,
rsfmri_8,
rsfmri_9,
rsfmri_10,
rsfmri_11,
rsfmri_12,
rsfmri_13,
rsfmri_14,
rsfmri_15,
rsfmri_16,
rsfmri_17,
rsfmri_18,
rsfmri_19,
rsfmri_20,
rsfmri_21,
rsfmri_22,
rsfmri_23,
rsfmri_24,

tfmri_1,
tfmri_2,
tfmri_5,

tfmri_c_1,
tfmri_c_2,
tfmri_c_5,

tracts,

tbss_FA_s,
tbss_ICVF_s,
tbss_ISOVF_s,
tbss_L1_s,
tbss_L2_s,
tbss_L3_s,
tbss_MD_s,
tbss_MO_s,
tbss_OD_s,

tbss_FA,
tbss_ICVF,
tbss_ISOVF,
tbss_L1,
tbss_L2,
tbss_L3,
tbss_MD,
tbss_MO,
tbss_OD
]

names = [

'T1_nonlinear',
'T1_linear',
'jacobian',
'vbm',

'T2_nonlinear',
'T2_lesions',
'swi',

'rsfmri_0',
'rsfmri_1',
'rsfmri_2',
'rsfmri_3',
'rsfmri_4',
'rsfmri_5',
'rsfmri_6',
'rsfmri_7',
'rsfmri_8',
'rsfmri_9',
'rsfmri_10',
'rsfmri_11',
'rsfmri_12',
'rsfmri_13',
'rsfmri_14',
'rsfmri_15',
'rsfmri_16',
'rsfmri_17',
'rsfmri_18',
'rsfmri_19',
'rsfmri_20',
'rsfmri_21',
'rsfmri_22',
'rsfmri_23',
'rsfmri_24',

'tfmri_1',
'tfmri_2',
'tfmri_5',

'tfmri_c_1',
'tfmri_c_2',
'tfmri_c_5',

'tracts',

'tbss_FA_s',
'tbss_ICVF_s',
'tbss_ISOVF_s',
'tbss_L1_s',
'tbss_L2_s',
'tbss_L3_s',
'tbss_MD_s',
'tbss_MO_s',
'tbss_OD_s',

'tbss_FA',
'tbss_ICVF',
'tbss_ISOVF',
'tbss_L1',
'tbss_L2',
'tbss_L3',
'tbss_MD',
'tbss_MO',
'tbss_OD'
]

dfs1, dfs2, dfs3 = [], [], []

dfs1 = dfs.copy()
dfs2 = dfs.copy()
dfs3 = dfs.copy()

for idx in range(len(dfs)):
    
    dfs1[idx] = '../predictions/AM2-' + dfs1[idx] + '_predictions/output_statistics_train.csv'
    dfs2[idx] = '../predictions/AM3-' + dfs2[idx] + '_predictions/output_statistics_train.csv'
    dfs3[idx] = '../predictions/AM4-' + dfs3[idx] + '_predictions/output_statistics_train.csv'
    
for idx in range(len(dfs)):
    dfs1[idx] = pd.read_csv(dfs1[idx])
    dfs2[idx] = pd.read_csv(dfs2[idx])
    dfs3[idx] = pd.read_csv(dfs3[idx])

In [4]:
for idx in range(len(dfs)):
    dfs[idx] = pd.merge(dfs1[idx], dfs2[idx], on='Unnamed: 0', how='inner')
    dfs[idx] = pd.merge(dfs[idx], dfs3[idx], on='Unnamed: 0', how='inner')
    
    dfs[idx].rename({
        'target_age_x': 'target_age_original',
        'output_age_x': 'output_age_1',
        'age_delta_x': 'age_delta_1',
        'loss_x': 'loss_1',
        'target_age_y': 'target_age_2',
        'output_age_y': 'output_age_2',
        'age_delta_y': 'age_delta_2',
        'loss_y': 'loss_2',
        'target_age': 'target_age_3',
        'output_age': 'output_age_3',
        'age_delta': 'age_delta_3',
        'loss': 'loss_3',
    }, axis=1, inplace=True)
    
    dfs[idx].drop(['target_age_2', 'target_age_3'], axis=1, inplace=True)
    
    dfs[idx].rename({
        'target_age_original': 'target_age',
    }, axis=1, inplace=True)
    
    dfs[idx]['mean_output_age'] = dfs[idx][['output_age_1', 'output_age_2', 'output_age_3']].mean(axis=1)
    dfs[idx]['mean_age_delta'] = dfs[idx]['mean_output_age'] - dfs[idx]['target_age']

In [5]:
for idx in range(len(dfs)):    
    dfs[idx]['output_age_decon'] = age_debiasing(dfs[idx].mean_output_age, dfs[idx].target_age) 
    dfs[idx]['age_delta_decon'] = dfs[idx].output_age_decon - dfs[idx].target_age

In [6]:
print("{:13} : {:15}  |  {:13}  |  {:13}  |  {:13}".format('Modality', 'Run1', "Run2", "Run3", "MEAN"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} : {:7.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  {:.3f} ± {:.3f}  |  MEAN --- {:.3f} ± {:.3f}".format(
        names[idx],
        np.abs(dfs[idx].age_delta_1).mean(), np.abs(dfs[idx].age_delta_1).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_2).mean(), np.abs(dfs[idx].age_delta_2).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_3).mean(), np.abs(dfs[idx].age_delta_3).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : Run1             |  Run2           |  Run3           |  MEAN         
-----------------------------------------------------------------------------------------------
T1_nonlinear  :   1.157 ± 0.009  |  0.682 ± 0.005  |  1.009 ± 0.008  |  MEAN --- 0.641 ± 0.005
T1_linear     :   0.615 ± 0.005  |  1.302 ± 0.010  |  1.073 ± 0.008  |  MEAN --- 0.678 ± 0.005
jacobian      :   0.468 ± 0.004  |  2.540 ± 0.020  |  2.483 ± 0.019  |  MEAN --- 1.518 ± 0.012
vbm           :   1.754 ± 0.014  |  2.095 ± 0.016  |  1.881 ± 0.015  |  MEAN --- 1.683 ± 0.013
T2_nonlinear  :   1.556 ± 0.012  |  1.516 ± 0.012  |  1.877 ± 0.015  |  MEAN --- 1.452 ± 0.011
T2_lesions    :   3.486 ± 0.027  |  3.847 ± 0.029  |  3.559 ± 0.028  |  MEAN --- 3.523 ± 0.027
swi           :   0.799 ± 0.006  |  2.691 ± 0.021  |  1.456 ± 0.012  |  MEAN --- 1.212 ± 0.010
rsfmri_0      :   3.386 ± 0.026  |  3.042 ± 0.024  |  3.527 ± 0.027  |  MEAN --- 3.052 ± 0.024
rsfmri_1      :   3.498 ± 0.027  |  3.874 ± 0.029  |  3.62

In [7]:
print("{:13} : {:14}  |  {:13}  ".format('Modality', 'MEAN (MAE)', "MEAN DECONF"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
    )
         )

Modality      : MEAN (MAE)      |  MEAN DECONF    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.641 ± 0.005  |   0.638 ± 0.005 
T1_linear     :  0.678 ± 0.005  |   0.652 ± 0.005 
jacobian      :  1.518 ± 0.012  |   1.484 ± 0.012 
vbm           :  1.683 ± 0.013  |   1.738 ± 0.013 
T2_nonlinear  :  1.452 ± 0.011  |   1.434 ± 0.011 
T2_lesions    :  3.523 ± 0.027  |   4.325 ± 0.033 
swi           :  1.212 ± 0.010  |   1.200 ± 0.009 
rsfmri_0      :  3.052 ± 0.024  |   3.407 ± 0.026 
rsfmri_1      :  3.393 ± 0.026  |   4.010 ± 0.032 
rsfmri_2      :  3.332 ± 0.026  |   3.934 ± 0.031 
rsfmri_3      :  3.576 ± 0.027  |   4.321 ± 0.034 
rsfmri_4      :  3.352 ± 0.026  |   3.808 ± 0.030 
rsfmri_5      :  3.473 ± 0.027  |   4.182 ± 0.033 
rsfmri_6      :  3.384 ± 0.025  |   3.932 ± 0.030 
rsfmri_7      :  2.892 ± 0.022  |   3.131 ± 0.024 
rsfmri_8      :  3.525 ± 0.027  |   4.083 ± 0.032 
rsfmri_9      :  3.726 ± 0.028  |   4

In [8]:
mae_weighting = 37.137671232876755 # for female subjects
# mae_weighting = 36.28150684931529 # for male subjects


print("{:13} : {:14}  |  {:15}  |  {:7}  |  {:13} ".format('Modality', 'MAE', "RMSE", 
                                                            "CORR (R)", "MAE weighted"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:5.3f} ".format(
        names[idx],
        np.abs(dfs[idx].mean_age_delta).mean(), np.abs(dfs[idx].mean_age_delta).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0],
        np.abs(dfs[idx].mean_age_delta).mean() / mae_weighting,
    )
         )

Modality      : MAE             |  RMSE             |  CORR (R)  |  MAE weighted  
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.641 ± 0.005  |   0.807 ± 0.098   |   0.994   |   0.017 
T1_linear     :  0.678 ± 0.005  |   0.851 ± 0.103   |   0.994   |   0.018 
jacobian      :  1.518 ± 0.012  |   1.906 ± 0.233   |   0.969   |   0.041 
vbm           :  1.683 ± 0.013  |   2.108 ± 0.256   |   0.959   |   0.045 
T2_nonlinear  :  1.452 ± 0.011  |   1.822 ± 0.222   |   0.972   |   0.039 
T2_lesions    :  3.523 ± 0.027  |   4.403 ± 0.526   |   0.806   |   0.095 
swi           :  1.212 ± 0.010  |   1.526 ± 0.186   |   0.980   |   0.033 
rsfmri_0      :  3.052 ± 0.024  |   3.811 ± 0.455   |   0.865   |   0.082 
rsfmri_1      :  3.393 ± 0.026  |   4.232 ± 0.506   |   0.824   |   0.091 
rsfmri_2      :  3.332 ± 0.026  |   4.162 ± 0.501   |   0.830   |   0.090 
rsfmri_3      :  3.576 ± 0.027  |   4.442 ± 0.528   |   0.805   |   0.0

In [9]:
maes=[]
prs = []
for idx in range(len(dfs)):
    mae = np.abs(dfs[idx].mean_age_delta).mean()
    maes.append(mae)
    pr = pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age)[0]
    prs.append(pr)
    
print(pearsonr(maes, prs)[0])

-0.9838023949155232


In [10]:
print("{:13} : {:14}  |  {:15}  |  {:13} ".format('Modality', 'MAE (DECON)', "RMSE (DECON)", "PR (DECON)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ".format(
        names[idx],
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean()), np.sqrt(np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx]))),
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  RMSE (DECON)     |  PR (DECON)    
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.638 ± 0.005  |   0.801 ± 0.097   |   0.994 
T1_linear     :  0.652 ± 0.005  |   0.821 ± 0.100   |   0.994 
jacobian      :  1.484 ± 0.012  |   1.874 ± 0.232   |   0.969 
vbm           :  1.738 ± 0.013  |   2.171 ± 0.260   |   0.959 
T2_nonlinear  :  1.434 ± 0.011  |   1.792 ± 0.216   |   0.972 
T2_lesions    :  4.325 ± 0.033  |   5.403 ± 0.647   |   0.806 
swi           :  1.200 ± 0.009  |   1.502 ± 0.182   |   0.980 
rsfmri_0      :  3.407 ± 0.026  |   4.257 ± 0.512   |   0.865 
rsfmri_1      :  4.010 ± 0.032  |   5.049 ± 0.612   |   0.824 
rsfmri_2      :  3.934 ± 0.031  |   4.934 ± 0.601   |   0.830 
rsfmri_3      :  4.321 ± 0.034  |   5.423 ± 0.658   |   0.805 
rsfmri_4      :  3.808 ± 0.030  |   4.768 ± 0.577   |   0.839 
rsfmri_5      :  4.182 ± 0.033  |   5.265 ± 0.640   |   0.813 
rsfmri_6      :

# Saving Dataframe for Later Use

In [11]:
d = {'modality':names, 'dataframe':dfs}
df = pd.DataFrame(data=d)

In [12]:
df['dataframe'].iloc[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,25868876,70.387295,72.681980,2.294681,5.265562,71.523300,1.136005,1.290508,69.981120,-0.406178,0.164980,71.395467,1.008172,71.563246,1.175951
1,21642613,62.258562,62.747707,0.489146,0.239264,62.648983,0.390421,0.152429,61.509250,-0.749311,0.561467,62.301980,0.043418,62.211033,-0.047528
2,24355695,58.123174,58.637386,0.514213,0.264415,56.885147,-1.238026,1.532709,59.448463,1.325290,1.756393,58.323665,0.200492,58.119529,-0.003645
3,21773603,59.247603,57.777943,-1.469660,2.159901,60.258280,1.010679,1.021472,60.074047,0.826444,0.683010,59.370090,0.122487,59.195726,-0.051877
4,21259046,61.839840,62.653072,0.813232,0.661347,61.794030,-0.045811,0.002099,63.790737,1.950897,3.805999,62.745946,0.906106,62.667631,0.827791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9409,22626616,68.528196,66.981440,-1.546759,2.392462,70.015230,1.487032,2.211264,70.186930,1.658731,2.751390,69.061200,0.533004,69.162565,0.634369
9410,22767103,68.792808,69.802870,1.010063,1.020228,67.590675,-1.202133,1.445123,67.882900,-0.909912,0.827939,68.425482,-0.367327,68.508759,-0.284049
9411,22510818,54.735274,53.012780,-1.722495,2.966988,54.591064,-0.144210,0.020796,53.512897,-1.222376,1.494204,53.705580,-1.029694,53.370051,-1.365223
9412,23314468,71.933219,70.737390,-1.195831,1.430011,72.153500,0.220284,0.048525,71.967730,0.034508,0.001191,71.619540,-0.313679,71.793694,-0.139525


In [13]:
df

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [14]:
type(df.dataframe.iloc[0])

pandas.core.frame.DataFrame

In [15]:
df.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,25868876,70.387295,72.681980,2.294681,5.265562,71.523300,1.136005,1.290508,69.981120,-0.406178,0.164980,71.395467,1.008172,71.563246,1.175951
1,21642613,62.258562,62.747707,0.489146,0.239264,62.648983,0.390421,0.152429,61.509250,-0.749311,0.561467,62.301980,0.043418,62.211033,-0.047528
2,24355695,58.123174,58.637386,0.514213,0.264415,56.885147,-1.238026,1.532709,59.448463,1.325290,1.756393,58.323665,0.200492,58.119529,-0.003645
3,21773603,59.247603,57.777943,-1.469660,2.159901,60.258280,1.010679,1.021472,60.074047,0.826444,0.683010,59.370090,0.122487,59.195726,-0.051877
4,21259046,61.839840,62.653072,0.813232,0.661347,61.794030,-0.045811,0.002099,63.790737,1.950897,3.805999,62.745946,0.906106,62.667631,0.827791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9409,22626616,68.528196,66.981440,-1.546759,2.392462,70.015230,1.487032,2.211264,70.186930,1.658731,2.751390,69.061200,0.533004,69.162565,0.634369
9410,22767103,68.792808,69.802870,1.010063,1.020228,67.590675,-1.202133,1.445123,67.882900,-0.909912,0.827939,68.425482,-0.367327,68.508759,-0.284049
9411,22510818,54.735274,53.012780,-1.722495,2.966988,54.591064,-0.144210,0.020796,53.512897,-1.222376,1.494204,53.705580,-1.029694,53.370051,-1.365223
9412,23314468,71.933219,70.737390,-1.195831,1.430011,72.153500,0.220284,0.048525,71.967730,0.034508,0.001191,71.619540,-0.313679,71.793694,-0.139525


In [16]:
df.iloc[0].modality

'T1_nonlinear'

In [17]:
type(df.iloc[0].dataframe)

pandas.core.frame.DataFrame

In [18]:
df.to_pickle('female_train.pkl', protocol = 4)

In [19]:
df2 = pd.read_pickle('female_train.pkl')

In [20]:
df2

,modality,dataframe
0,T1_nonlinear,Unnamed: 0 target_age output_age_1 ag...
1,T1_linear,Unnamed: 0 target_age output_age_1 ag...
2,jacobian,Unnamed: 0 target_age output_age_1 ag...
3,vbm,Unnamed: 0 target_age output_age_1 ag...
4,T2_nonlinear,Unnamed: 0 target_age output_age_1 ag...
5,T2_lesions,Unnamed: 0 target_age output_age_1 ag...
6,swi,Unnamed: 0 target_age output_age_1 ag...
7,rsfmri_0,Unnamed: 0 target_age output_age_1 ag...
8,rsfmri_1,Unnamed: 0 target_age output_age_1 ag...
9,rsfmri_2,Unnamed: 0 target_age output_age_1 ag...


In [21]:
df2.iloc[0].dataframe

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,25868876,70.387295,72.681980,2.294681,5.265562,71.523300,1.136005,1.290508,69.981120,-0.406178,0.164980,71.395467,1.008172,71.563246,1.175951
1,21642613,62.258562,62.747707,0.489146,0.239264,62.648983,0.390421,0.152429,61.509250,-0.749311,0.561467,62.301980,0.043418,62.211033,-0.047528
2,24355695,58.123174,58.637386,0.514213,0.264415,56.885147,-1.238026,1.532709,59.448463,1.325290,1.756393,58.323665,0.200492,58.119529,-0.003645
3,21773603,59.247603,57.777943,-1.469660,2.159901,60.258280,1.010679,1.021472,60.074047,0.826444,0.683010,59.370090,0.122487,59.195726,-0.051877
4,21259046,61.839840,62.653072,0.813232,0.661347,61.794030,-0.045811,0.002099,63.790737,1.950897,3.805999,62.745946,0.906106,62.667631,0.827791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9409,22626616,68.528196,66.981440,-1.546759,2.392462,70.015230,1.487032,2.211264,70.186930,1.658731,2.751390,69.061200,0.533004,69.162565,0.634369
9410,22767103,68.792808,69.802870,1.010063,1.020228,67.590675,-1.202133,1.445123,67.882900,-0.909912,0.827939,68.425482,-0.367327,68.508759,-0.284049
9411,22510818,54.735274,53.012780,-1.722495,2.966988,54.591064,-0.144210,0.020796,53.512897,-1.222376,1.494204,53.705580,-1.029694,53.370051,-1.365223
9412,23314468,71.933219,70.737390,-1.195831,1.430011,72.153500,0.220284,0.048525,71.967730,0.034508,0.001191,71.619540,-0.313679,71.793694,-0.139525


In [23]:
df2.iloc[0].modality

'T1_nonlinear'

In [24]:
type(df2.iloc[0].dataframe)

pandas.core.frame.DataFrame

# Test what happens if debiasing before deconfounding

In [25]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon
0,25868876,70.387295,72.681980,2.294681,5.265562,71.523300,1.136005,1.290508,69.981120,-0.406178,0.164980,71.395467,1.008172,71.563246,1.175951
1,21642613,62.258562,62.747707,0.489146,0.239264,62.648983,0.390421,0.152429,61.509250,-0.749311,0.561467,62.301980,0.043418,62.211033,-0.047528
2,24355695,58.123174,58.637386,0.514213,0.264415,56.885147,-1.238026,1.532709,59.448463,1.325290,1.756393,58.323665,0.200492,58.119529,-0.003645
3,21773603,59.247603,57.777943,-1.469660,2.159901,60.258280,1.010679,1.021472,60.074047,0.826444,0.683010,59.370090,0.122487,59.195726,-0.051877
4,21259046,61.839840,62.653072,0.813232,0.661347,61.794030,-0.045811,0.002099,63.790737,1.950897,3.805999,62.745946,0.906106,62.667631,0.827791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9409,22626616,68.528196,66.981440,-1.546759,2.392462,70.015230,1.487032,2.211264,70.186930,1.658731,2.751390,69.061200,0.533004,69.162565,0.634369
9410,22767103,68.792808,69.802870,1.010063,1.020228,67.590675,-1.202133,1.445123,67.882900,-0.909912,0.827939,68.425482,-0.367327,68.508759,-0.284049
9411,22510818,54.735274,53.012780,-1.722495,2.966988,54.591064,-0.144210,0.020796,53.512897,-1.222376,1.494204,53.705580,-1.029694,53.370051,-1.365223
9412,23314468,71.933219,70.737390,-1.195831,1.430011,72.153500,0.220284,0.048525,71.967730,0.034508,0.001191,71.619540,-0.313679,71.793694,-0.139525


In [28]:
for idx in range(len(dfs)):
    dfs[idx]['output_age_decon_1'] = age_debiasing(dfs[idx].output_age_1, dfs[idx].target_age)
    dfs[idx]['output_age_decon_2'] = age_debiasing(dfs[idx].output_age_2, dfs[idx].target_age) 
    dfs[idx]['output_age_decon_3'] = age_debiasing(dfs[idx].output_age_3, dfs[idx].target_age) 
    dfs[idx]['mean_output_age_decon'] = dfs[idx][['output_age_decon_1', 'output_age_decon_2', 'output_age_decon_3']].mean(axis=1)
    dfs[idx]['mean_age_delta_decon'] = dfs[idx]['mean_output_age_decon'] - dfs[idx]['target_age']

In [29]:
dfs[0]

,Unnamed: 0,target_age,output_age_1,age_delta_1,loss_1,output_age_2,age_delta_2,loss_2,output_age_3,age_delta_3,loss_3,mean_output_age,mean_age_delta,output_age_decon,age_delta_decon,output_age_decon_1,output_age_decon_2,output_age_decon_3,mean_output_age_decon,mean_age_delta_decon
0,25868876,70.387295,72.681980,2.294681,5.265562,71.523300,1.136005,1.290508,69.981120,-0.406178,0.164980,71.395467,1.008172,71.563246,1.175951,72.980680,71.342884,70.343960,71.555841,1.168546
1,21642613,62.258562,62.747707,0.489146,0.239264,62.648983,0.390421,0.152429,61.509250,-0.749311,0.561467,62.301980,0.043418,62.211033,-0.047528,62.753028,62.438741,61.417188,62.202985,-0.055576
2,24355695,58.123174,58.637386,0.514213,0.264415,56.885147,-1.238026,1.532709,59.448463,1.325290,1.756393,58.323665,0.200492,58.119529,-0.003645,58.521321,56.655533,59.245745,58.140866,0.017693
3,21773603,59.247603,57.777943,-1.469660,2.159901,60.258280,1.010679,1.021472,60.074047,0.826444,0.683010,59.370090,0.122487,59.195726,-0.051877,57.636496,60.040003,59.904920,59.193807,-0.053796
4,21259046,61.839840,62.653072,0.813232,0.661347,61.794030,-0.045811,0.002099,63.790737,1.950897,3.805999,62.745946,0.906106,62.667631,0.827791,62.655598,61.580915,63.821180,62.685898,0.846057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9409,22626616,68.528196,66.981440,-1.546759,2.392462,70.015230,1.487032,2.211264,70.186930,1.658731,2.751390,69.061200,0.533004,69.162565,0.634369,67.111791,69.829746,70.560821,69.167453,0.639256
9410,22767103,68.792808,69.802870,1.010063,1.020228,67.590675,-1.202133,1.445123,67.882900,-0.909912,0.827939,68.425482,-0.367327,68.508759,-0.284049,70.016544,67.397042,68.133075,68.515553,-0.277255
9411,22510818,54.735274,53.012780,-1.722495,2.966988,54.591064,-0.144210,0.020796,53.512897,-1.222376,1.494204,53.705580,-1.029694,53.370051,-1.365223,52.730608,54.353740,52.991465,53.358605,-1.376669
9412,23314468,71.933219,70.737390,-1.195831,1.430011,72.153500,0.220284,0.048525,71.967730,0.034508,0.001191,71.619540,-0.313679,71.793694,-0.139525,70.978662,71.975202,72.437242,71.797036,-0.136184


In [30]:
print("{:13} : {:14}  |  {:14}  |  {:13}  |   {:13}  |  {:}  |  {:} ".format('Modality', 'MAE (DECON)', 'MAE (DECON B)', "MSE (DECON)", "MSE (DECON B)",  "PR (DECON)", "PR (DECON B)"))
print('-----------------------------------------------------------------------------------------------')

for idx in range(len(dfs)):
    print("{:13} :  {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f} ± {:.3f}  |   {:.3f} ± {:.3f}   |   {:.3f}   |   {:.3f} ".format(
        names[idx],
        
        np.abs(dfs[idx].age_delta_decon).mean(), np.abs(dfs[idx].age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        np.abs(dfs[idx].mean_age_delta_decon).mean(), np.abs(dfs[idx].mean_age_delta_decon).std()/np.sqrt(len(dfs[idx])),
        
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).mean(), np.square(np.subtract(dfs[idx].target_age, dfs[idx].mean_output_age_decon)).std()/np.sqrt(len(dfs[idx])),
        
        pearsonr(dfs[idx].target_age, dfs[idx].output_age_decon)[0],
        pearsonr(dfs[idx].target_age, dfs[idx].mean_output_age_decon)[0],
    )
         )

Modality      : MAE (DECON)     |  MAE (DECON B)   |  MSE (DECON)    |   MSE (DECON B)  |  PR (DECON)  |  PR (DECON B) 
-----------------------------------------------------------------------------------------------
T1_nonlinear  :  0.638 ± 0.005  |   0.642 ± 0.005   |   0.641 ± 0.009  |   0.649 ± 0.009   |   0.994   |   0.994 
T1_linear     :  0.652 ± 0.005  |   0.658 ± 0.005   |   0.674 ± 0.010  |   0.686 ± 0.010   |   0.994   |   0.994 
jacobian      :  1.484 ± 0.012  |   1.639 ± 0.013   |   3.514 ± 0.054  |   4.286 ± 0.065   |   0.969   |   0.963 
vbm           :  1.738 ± 0.013  |   1.741 ± 0.013   |   4.714 ± 0.068  |   4.728 ± 0.068   |   0.959   |   0.959 
T2_nonlinear  :  1.434 ± 0.011  |   1.440 ± 0.011   |   3.212 ± 0.047  |   3.241 ± 0.047   |   0.972   |   0.971 
T2_lesions    :  4.325 ± 0.033  |   4.339 ± 0.033   |   29.193 ± 0.419  |   29.380 ± 0.422   |   0.806   |   0.805 
swi           :  1.200 ± 0.009  |   1.335 ± 0.010   |   2.255 ± 0.033  |   2.789 ± 0.041   |   0.9